# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.




Цель проекта, показать на реальных данных, что качество предсказания не ухудшится при использовании предложенных методов защиты персональных данных



    Ответьте на вопрос и обоснуйте решение.
    Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    a. Изменится. Приведите примеры матриц.
    b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Вариант "b"

Параметры в Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Следовательно, при умножении на обратную матрицу от исходной результат не должен измениться.

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$

 
    
    
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
\   
$$
w' = (P^T) ^{-1} (X^T X)^{-1} (P)^{-1} (XP)^T y
$$
\  
$$
w' =  (X^T X)^{-1}(X)^Ty  (P^T) ^{-1} (P)^{-1} (P)^T 
$$
\   
$$
w' =  (X^T X)^{-1}(X)^Ty  (P^T) ^{-1} (P)^{-1} (P)^T 
$$
\
$$
w' P=  (X^T X)^{-1}(X)^Ty  (P^T) ^{-1}  (P)^T 
$$
\
$$
w=  (X^T X)^{-1}(X)^Ty 
$$
    
</div>



$$w' = (P^T (X^T X) P)^{-1} (XP)^T y$$
$$w' = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y$$
$$w=  (X^T X)^{-1}(X)^Ty $$




## Загрузка данных

    Загрузим и изучим данные.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import os
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score

In [2]:
#pd.read_csv('ins.csv')  

pth1 = '/datasets/insurance.csv'
pth2 = 'ins.csv'
df = pd.read_csv(pth2)
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    pass
    print('Something is wrong')

In [3]:
df.sample(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
1239,1,24.0,33300.0,2,0
4808,0,23.0,50900.0,0,0
3109,0,35.0,48800.0,0,0
3099,1,34.0,53100.0,0,0
1477,0,20.0,33100.0,2,0
4474,0,41.0,46000.0,1,0
796,1,36.0,32300.0,2,0
3013,0,19.0,53500.0,2,0
2986,1,33.0,40200.0,1,0
791,1,34.0,31700.0,2,0


In [4]:
# посмотрим более детально на то, какие данные содержатся в каждом из столбцов данных
for col in df.columns:
    if not "sales" in col:
        print('---------------------------------'+col+'-----------------------------------------')

        print(df.groupby([col], dropna=False, as_index=False).size())

---------------------------------Пол-----------------------------------------
   Пол  size
0    0  2505
1    1  2495
---------------------------------Возраст-----------------------------------------
    Возраст  size
0      18.0   117
1      19.0   223
2      20.0   195
3      21.0   200
4      22.0   209
5      23.0   202
6      24.0   182
7      25.0   214
8      26.0   211
9      27.0   209
10     28.0   204
11     29.0   203
12     30.0   202
13     31.0   212
14     32.0   206
15     33.0   191
16     34.0   177
17     35.0   179
18     36.0   193
19     37.0   147
20     38.0   139
21     39.0   141
22     40.0   114
23     41.0   129
24     42.0    93
25     43.0    77
26     44.0    74
27     45.0    73
28     46.0    60
29     47.0    47
30     48.0    26
31     49.0    37
32     50.0    27
33     51.0    21
34     52.0    22
35     53.0    11
36     54.0     7
37     55.0     9
38     56.0     5
39     57.0     2
40     58.0     2
41     59.0     3
42     60.0     2
43     61

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
features = df.drop('Страховые выплаты',axis = 1).values
target = df['Страховые выплаты']
features.shape

(5000, 4)

## Алгоритм преобразования

    Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

Для преобразования данных используем умножение исходных данных на произвольную обратимую матрицу:

In [7]:
def X_modified(X, X_test, seed):
    try:
        state = np.random.RandomState(seed=seed)
        rand_matrix = state.normal(size=(X.shape[1], X.shape[1])) # Квадратная матрица P
        inv = np.linalg.inv(rand_matrix) # Проверка на обратимость
        x_modified = X @ rand_matrix # Умножаем признаки на обратимую матрицу
        x_test_modified = X_test @ rand_matrix
        return x_modified, inv, x_test_modified
    except np.linalg.LinAlgError:
        X_modified(X, X_test, seed) # Если возникла ошибка, заново запускаем нашу функцию
        


In [8]:
results=[]

## Проверка алгоритма

    Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [9]:
def get_r2_linear(train_features,test_features,train_target, test_target):
    reg = LinearRegression().fit(train_features, train_target)
    predicts = reg.predict(test_features)
    print("r2_score( test_target, predicts)=",r2_score(test_target, predicts))

Определим коэффициент детерминации на исходных данных:

In [10]:
features_train, features_test, target_train, target_test = train_test_split(
   features, target, test_size=0.20, random_state=12345)


In [11]:
get_r2_linear(features_train, features_test, target_train, target_test)

r2_score( test_target, predicts)= 0.41176839567705226


Подготовим данные для расчета R2 после преобразования:

In [12]:
features_new, inv_P,features_new_test = X_modified(features_train,features_test, 12345)

In [13]:
get_r2_linear(features_new, features_new_test, target_train, target_test)


r2_score( test_target, predicts)= 0.4117683956770183


In [14]:
features_train

array([[0.00e+00, 2.00e+01, 3.98e+04, 2.00e+00],
       [1.00e+00, 4.00e+01, 3.42e+04, 0.00e+00],
       [1.00e+00, 4.50e+01, 5.08e+04, 1.00e+00],
       ...,
       [1.00e+00, 4.10e+01, 4.47e+04, 1.00e+00],
       [0.00e+00, 2.20e+01, 5.01e+04, 4.00e+00],
       [0.00e+00, 1.90e+01, 4.08e+04, 0.00e+00]])

In [15]:
features_new

array([[ 30646.96378567,  49636.42845059,  40090.70042301,
        -51582.19236093],
       [ 26378.99832757,  42684.28316107,  34449.07290569,
        -44320.04781583],
       [ 39154.87684442,  63382.29572712,  51170.23369779,
        -65835.02306699],
       ...,
       [ 34455.9760597 ,  55773.47021184,  45026.00698503,
        -57929.20128602],
       [ 38572.37777648,  62477.95087279,  50467.64245554,
        -64930.9334338 ],
       [ 31413.47058944,  50881.01195539,  41095.09103904,
        -52880.46807488]])

Функция детерменации для исходного и преобразованного датасета совпадает до 11 знака. Следовательно, данный метод можно использовать при защите данных, качество машинного обучения и предсказания от этого не изменится.